In [1]:
import cubewalkers as cw
import cupy as cp
from copy import deepcopy

In [2]:
IMPORT_RULES_FROM_FILES = True
cc_models_dir = './models/cell_collective/'
if IMPORT_RULES_FROM_FILES:
    from os import listdir
    
    sync_models = {}
    for fname in listdir(cc_models_dir):
        with open(cc_models_dir+fname) as rulefile:
            name = fname.strip('.txt')
            rules = rulefile.read()
            sync_models[name]=cw.Model(rules)
else:
    from cana.datasets.bio import load_all_cell_collective_models
    def cell_collective_models():
        return {BN.name:cw.Model(cw.conversions.network_rules_from_cana(BN)) 
                for BN in load_all_cell_collective_models()}
    sync_models = cell_collective_models()
    for name,model in sync_models.items():
        with open(cc_models_dir+name+'.txt','w') as rulefile:
            rulefile.write(model.rules)

total_models = len(sync_models)
async_models = deepcopy(sync_models)

In [3]:
def simulate_models(models, sync=True ,W=2500):
    convergence_measures = {}
    for model_idx, (model_name, model) in enumerate(models.items()):
        model.n_walkers = W
        N = model.n_variables
        timescale=2*N
        T = N**2 + 5 * timescale
        T_window = 5 * timescale
        model.n_time_steps = T

        print(f"Simulating Model {model_name} ({W=},{T=},{N=}). . .")
        if sync:
            model.simulate_ensemble(T_window=T_window,
                                    averages_only=True,
                                    maskfunction=cw.update_schemes.synchronous,
                                    threads_per_block=(16, 16))
        else:
            model.simulate_ensemble(T_window=T_window,
                                    averages_only=True,
                                    maskfunction=cw.update_schemes.asynchronous,
                                    threads_per_block=(16, 16))
            
        tw1 = model.trajectories[0*timescale:2*timescale]
        tw2 = model.trajectories[1*timescale:3*timescale]
        tw3 = model.trajectories[2*timescale:4*timescale]
        tw4 = model.trajectories[3*timescale:]
        convergence_measures[model_name] = max([
            cp.max(cp.abs(tw1.mean(axis=0) - tw2.mean(axis=0))),    
            cp.max(cp.abs(tw1.mean(axis=0) - tw3.mean(axis=0))),
            cp.max(cp.abs(tw1.mean(axis=0) - tw4.mean(axis=0))),
            cp.max(cp.abs(tw2.mean(axis=0) - tw3.mean(axis=0))),
            cp.max(cp.abs(tw2.mean(axis=0) - tw4.mean(axis=0))),
            cp.max(cp.abs(tw3.mean(axis=0) - tw4.mean(axis=0))),
            ])
        print(f"Progress: {(model_idx+1)}/{total_models},\t maximum difference: {convergence_measures[model_name]}")
    return convergence_measures

In [4]:
sync_convergence_measures=simulate_models(sync_models,sync=True)

Simulating Model T-LGL Survival Network 2011 Reduced Network (W=2500,T=504,N=18). . .
Progress: 1/74,	 maximum difference: 0.0
Simulating Model TOL Regulatory Network (W=2500,T=816,N=24). . .
Progress: 2/74,	 maximum difference: 0.0
Simulating Model B cell differentiation (W=2500,T=704,N=22). . .
Progress: 3/74,	 maximum difference: 0.0
Simulating Model Death Receptor Signaling (W=2500,T=1064,N=28). . .
Progress: 4/74,	 maximum difference: 0.0005178571428572365
Simulating Model HCC1954 Breast Cell Line Short-term ErbB Network (W=2500,T=416,N=16). . .
Progress: 5/74,	 maximum difference: 0.0
Simulating Model IL-1 Signaling (W=2500,T=15104,N=118). . .
Progress: 6/74,	 maximum difference: 0.0
Simulating Model T-LGL Survival Network 2011 (W=2500,T=4200,N=60). . .
Progress: 7/74,	 maximum difference: 0.0
Simulating Model Mammalian Cell Cycle (W=2500,T=600,N=20). . .
Progress: 8/74,	 maximum difference: 0.0
Simulating Model Signaling in Macrophage Activation (W=2500,T=106251,N=321). . .
Prog

In [5]:
async_convergence_measures=simulate_models(async_models,sync=False)

Simulating Model T-LGL Survival Network 2011 Reduced Network (W=2500,T=504,N=18). . .
Progress: 1/74,	 maximum difference: 0.002472222222222216
Simulating Model TOL Regulatory Network (W=2500,T=816,N=24). . .
Progress: 2/74,	 maximum difference: 0.0
Simulating Model B cell differentiation (W=2500,T=704,N=22). . .
Progress: 3/74,	 maximum difference: 0.0
Simulating Model Death Receptor Signaling (W=2500,T=1064,N=28). . .
Progress: 4/74,	 maximum difference: 0.0
Simulating Model HCC1954 Breast Cell Line Short-term ErbB Network (W=2500,T=416,N=16). . .
Progress: 5/74,	 maximum difference: 0.0
Simulating Model IL-1 Signaling (W=2500,T=15104,N=118). . .
Progress: 6/74,	 maximum difference: 0.0
Simulating Model T-LGL Survival Network 2011 (W=2500,T=4200,N=60). . .
Progress: 7/74,	 maximum difference: 0.0018450000000000012
Simulating Model Mammalian Cell Cycle (W=2500,T=600,N=20). . .
Progress: 8/74,	 maximum difference: 0.0
Simulating Model Signaling in Macrophage Activation (W=2500,T=106251

In [8]:
print(f'{max(sync_convergence_measures.values())}')
print(f'{max(async_convergence_measures.values())}')

0.03770555555555544
0.04071000000000008


In [9]:
with open('./data/converged_average_node_values.csv','w') as f:
    for model_name, smodel, amodel in [(k,sync_models[k],async_models[k]) for k in sorted(async_models)]:
        straj = ','.join(map(lambda x: str(cp.round(x,3)),cp.mean(smodel.trajectories,axis=0)))
        atraj = ','.join(map(lambda x: str(cp.round(x,3)),cp.mean(amodel.trajectories,axis=0)))
        f.write(f'{model_name},{cp.round(sync_convergence_measures[model_name],3)},synchronous,{straj}\n')
        f.write(f'{model_name},{cp.round(async_convergence_measures[model_name],3)},asynchronous,{atraj}\n')
    